In [60]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
from Utils.DataLoader import DataLoader
from Utils.DataTransformer import DataTransformer
from Utils.DuckDb.DuckDb import DuckDb
from Utils.DatasetProcessor.DatasetProcessorUtils import DatasetProcessorUtils
from Utils.DataAnalysis import DataAnalysis
from datetime import datetime
import pandas as pd
from typing import List
from Utils import PandasNotebookConfigs
import project_config
import numpy as np
import matplotlib.pyplot as plt
import joblib
from datetime import datetime
from Utils.Datetime import DatetimeUtils
from Utils.Dict import DictUtils
from Utils.Number import NumberUtils

In [62]:
PandasNotebookConfigs.config()

Ajustando display.float.format para %.4f
Ajustando prints de linhas e colunas


In [63]:
# 50% of the users will renew the plan
SUCESS_USER_PROACTIVE_ACTION = 0.50

COST_TO_KEEP_CUSTOMER = 200 # Mean paid price per month

RETAIN_MONTHS_QTY = 12

In [64]:
DUCK_DB = DuckDb()
DATA_TRANSFORMER = DataTransformer()
DATA_PROCESSOR_UTILS = DatasetProcessorUtils(DUCK_DB)
DATETIME_UTILS = DatetimeUtils()
DICT_UTILS = DictUtils()
DATA_ANALYSIS = DataAnalysis()
NUMBER_UTILS = NumberUtils()

In [65]:
df = DUCK_DB.load_table(
    project_config.PREDICTED_DATA_TABLE
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [66]:
len(df)

1251070

In [67]:
# Get only data to predict
# df = df[df['no_churn_information'] == True]
len(df)

1251070

In [68]:
# Get only users from the last month
max_safra = df['safra'].max() - 0
print(f'Registros da safra de {max_safra}')

df = df[df['safra'] == max_safra]
len(df)

Registros da safra de 201702


173975

In [69]:
df.head()

,cost,msno,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours,msno_1,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,safra_1,transaction_date_year,transaction_date_month,transaction_date_day,transaction_date_day_of_week,transaction_date_day_of_year,membership_expire_date_year,membership_expire_date_month,membership_expire_date_day,membership_expire_date_day_of_week,membership_expire_date_day_of_year,discount,price_per_month,members_msno,members_safra,is_active,registration_init_time_year,registration_init_time_month,registration_init_time_day,registration_init_time_day_of_week,registration_init_time_day_of_year,_filled_out_members_info,is_churn,no_churn_information,num_unq-2M,num_unq-1M,total_secs-2M,total_secs-1M,num_25-2M,num_25-1M,num_50-2M,num_50-1M,num_75-2M,num_75-1M,num_985-2M,num_985-1M,num_100-2M,num_100-1M,remaining_days,_filled_out_past_info,payment_method_id_6,payment_method_id_8,payment_method_id_10,payment_method_id_11,payment_method_id_12,payment_method_id_13,payment_method_id_14,payment_method_id_15,payment_method_id_16,payment_method_id_17,payment_method_id_18,payment_method_id_19,payment_method_id_20,payment_method_id_21,payment_method_id_22,payment_method_id_23,payment_method_id_26,payment_method_id_27,payment_method_id_28,payment_method_id_29,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_40,payment_method_id_41,city_1,city_10,city_11,city_12,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22,city_3,city_4,city_5,city_6,city_7,city_8,city_9,registered_via_13,registered_via_3,registered_via_4,registered_via_7,registered_via_9,payment_method_id_0,payment_method_id_1,payment_method_id_2,payment_method_id_3,payment_method_id_4,payment_method_id_5,payment_method_id_7,payment_method_id_9,payment_method_id_24,payment_method_id_25,city_0,city_2,registered_via_0,registered_via_1,registered_via_2,registered_via_5,registered_via_6,registered_via_8,registered_via_10,registered_via_11,registered_via_12,registered_via_14,registered_via_15,registered_via_16,registered_via_17,registered_via_18,registered_via_19,safra_year,safra_month,predicted_is_churn,predicted_is_churn_proba_false,predicted_is_churn_proba_true,predict_certain
4,68.9275,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201702,116,61,58,114,468,642,156533,43.4814,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30,99,99,True,False,201702,2017,2,15,2,46,2017,3,15,2,74,0,99.0000,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,<NA>,True,2014,6,8,6,159,Copiadas,False,True,979.0000,696.0000,256812.0000,157868.0000,193.0000,135.0000,88.0000,69.0000,98.0000,78.0000,142.0000,147.0000,855.0000,438.0000,28.0000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2017.0000,2.0000,False,0.6513,0.3487,0.6513
15,65.1869,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,201702,188,80,46,46,383,551,123768,34.3800,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,30,149,149,True,False,201702,2017,2,26,6,57,2017,3,26,6,85,0,149.0000,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,<NA>,True,2014,9,27,5,270,Copiadas,False,True,645.0000,1088.0000,169086.0000,215757.0000,166.0000,289.0000,73.0000,175.0000,49.0000,97.0000,43.0000,82.0000,579.0000,651.0000,28.0000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [70]:
# users_df = df.groupby('msno').tail(1)
users_df = df.copy()

In [71]:
users_df.head()

,cost,msno,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,total_hours,msno_1,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,is_cancel,safra_1,transaction_date_year,transaction_date_month,transaction_date_day,transaction_date_day_of_week,transaction_date_day_of_year,membership_expire_date_year,membership_expire_date_month,membership_expire_date_day,membership_expire_date_day_of_week,membership_expire_date_day_of_year,discount,price_per_month,members_msno,members_safra,is_active,registration_init_time_year,registration_init_time_month,registration_init_time_day,registration_init_time_day_of_week,registration_init_time_day_of_year,_filled_out_members_info,is_churn,no_churn_information,num_unq-2M,num_unq-1M,total_secs-2M,total_secs-1M,num_25-2M,num_25-1M,num_50-2M,num_50-1M,num_75-2M,num_75-1M,num_985-2M,num_985-1M,num_100-2M,num_100-1M,remaining_days,_filled_out_past_info,payment_method_id_6,payment_method_id_8,payment_method_id_10,payment_method_id_11,payment_method_id_12,payment_method_id_13,payment_method_id_14,payment_method_id_15,payment_method_id_16,payment_method_id_17,payment_method_id_18,payment_method_id_19,payment_method_id_20,payment_method_id_21,payment_method_id_22,payment_method_id_23,payment_method_id_26,payment_method_id_27,payment_method_id_28,payment_method_id_29,payment_method_id_30,payment_method_id_31,payment_method_id_32,payment_method_id_33,payment_method_id_34,payment_method_id_35,payment_method_id_36,payment_method_id_37,payment_method_id_38,payment_method_id_39,payment_method_id_40,payment_method_id_41,city_1,city_10,city_11,city_12,city_13,city_14,city_15,city_16,city_17,city_18,city_19,city_20,city_21,city_22,city_3,city_4,city_5,city_6,city_7,city_8,city_9,registered_via_13,registered_via_3,registered_via_4,registered_via_7,registered_via_9,payment_method_id_0,payment_method_id_1,payment_method_id_2,payment_method_id_3,payment_method_id_4,payment_method_id_5,payment_method_id_7,payment_method_id_9,payment_method_id_24,payment_method_id_25,city_0,city_2,registered_via_0,registered_via_1,registered_via_2,registered_via_5,registered_via_6,registered_via_8,registered_via_10,registered_via_11,registered_via_12,registered_via_14,registered_via_15,registered_via_16,registered_via_17,registered_via_18,registered_via_19,safra_year,safra_month,predicted_is_churn,predicted_is_churn_proba_false,predicted_is_churn_proba_true,predict_certain
4,68.9275,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,201702,116,61,58,114,468,642,156533,43.4814,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,30,99,99,True,False,201702,2017,2,15,2,46,2017,3,15,2,74,0,99.0000,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,<NA>,True,2014,6,8,6,159,Copiadas,False,True,979.0000,696.0000,256812.0000,157868.0000,193.0000,135.0000,88.0000,69.0000,98.0000,78.0000,142.0000,147.0000,855.0000,438.0000,28.0000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,2017.0000,2.0000,False,0.6513,0.3487,0.6513
15,65.1869,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,201702,188,80,46,46,383,551,123768,34.3800,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,30,149,149,True,False,201702,2017,2,26,6,57,2017,3,26,6,85,0,149.0000,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,<NA>,True,2014,9,27,5,270,Copiadas,False,True,645.0000,1088.0000,169086.0000,215757.0000,166.0000,289.0000,73.0000,175.0000,49.0000,97.0000,43.0000,82.0000,579.0000,651.0000,28.0000,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [72]:
users_df['predicted_is_churn'].value_counts()

predicted_is_churn
False    119576
True      54399
Name: count, dtype: int64

In [73]:
predicted_is_churn_df = users_df[users_df['predicted_is_churn'] == True]
predicted_not_is_churn_df = users_df[users_df['predicted_is_churn'] == False]

false_percentage = len(predicted_not_is_churn_df) * 100 / len(df)
true_percentage = 100 - false_percentage

print(f'Qtd. total de registros: {len(df)}')
print(f'Qtd. de previsões de churn: {len(predicted_is_churn_df)} ({NUMBER_UTILS.format_float_number(true_percentage, 2)}%)')
print(f'Qtd. de previsões de NÃO churn: {len(predicted_not_is_churn_df)} ({NUMBER_UTILS.format_float_number(false_percentage, 2)}%)')

Qtd. total de registros: 173975
Qtd. de previsões de churn: 54399 (31,26%)
Qtd. de previsões de NÃO churn: 119576 (68,73%)


In [74]:
mean_price = users_df['price_per_month'].mean()
print(f'Preço médio pago por mês: ${mean_price:.2f}')

Preço médio pago por mês: $132.32


In [75]:
proactive_action_gross_income = len(predicted_is_churn_df) * SUCESS_USER_PROACTIVE_ACTION * mean_price * RETAIN_MONTHS_QTY
costs = len(predicted_is_churn_df) * COST_TO_KEEP_CUSTOMER
liquid_income = proactive_action_gross_income - costs

print(f'Total de usuários afetados pela ação: {NUMBER_UTILS.format_integer_number(len(predicted_is_churn_df))}')

print(f'Valor total arrecadado pela ação proativa: ${NUMBER_UTILS.format_float_number(proactive_action_gross_income, 2)}')
print(f'Custos totais: ${NUMBER_UTILS.format_float_number(costs)}')
print(f'Total líquido: ${NUMBER_UTILS.format_float_number(liquid_income, 2)}')

Total de usuários afetados pela ação: 54.399
Valor total arrecadado pela ação proativa: $43.189.863,02
Custos totais: $10.879.800
Total líquido: $32.310.063,02
